# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043350,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047274,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.136877,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.149792,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.219998,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 43222
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
def eval_model(dataset_train, dataset_test):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8748451053283767,
 'recall': 0.9388297872340425,
 'auroc': 0.7657858613589568,
 'dp': 0.15769354186140594,
 'ftu': 0.008499999999999952}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-02-04 16:55:26.745579: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-04 16:55:26.871962: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  2%|▏         | 1/50 [00:14<12:09, 14.89s/it]

0 [D loss: 0.000034, acc.: 100.00%] [G loss: 43.320641]
generated_data


  4%|▍         | 2/50 [00:28<11:25, 14.28s/it]

1 [D loss: 0.000001, acc.: 100.00%] [G loss: 71.679199]


  6%|▌         | 3/50 [00:42<11:08, 14.22s/it]

...

48 [D loss: 0.000000, acc.: 100.00%] [G loss: 359.248108]


100%|██████████| 50/50 [11:29<00:00, 13.80s/it]


49 [D loss: 0.000000, acc.: 100.00%] [G loss: 385.927917]


Synthetic data generation: 100%|██████████| 338/338 [00:03<00:00, 106.60it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.080582,0.666667,0.035735,0.333333,0.133333,0.333333,0.846154,0.6,0.00,0.0,0.017997,0.026124,0.096906,0.225,0.0
1,0.080582,0.333333,0.035735,0.333333,0.533333,0.333333,0.846154,0.6,0.50,1.0,0.017997,0.026124,0.096906,0.275,1.0
2,0.080582,0.833333,0.035735,0.066667,0.933333,0.666667,0.000000,0.8,0.25,1.0,0.017997,0.026124,0.096906,0.150,1.0
3,0.080582,0.166667,0.035735,0.466667,0.800000,0.166667,0.230769,0.4,0.00,0.0,0.017997,0.026124,0.096906,0.325,1.0
4,0.080582,0.833333,0.035735,0.400000,0.533333,0.500000,0.923077,0.2,0.50,0.0,0.017997,0.026124,0.096906,0.475,1.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.6971736204576043,
 'recall': 0.6888297872340425,
 'auroc': 0.39078586135895677,
 'dp': 0.7117996939976097,
 'ftu': 0.47}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

  2%|▏         | 1/50 [00:05<04:48,  5.88s/it]

Epoch: 0 | disc_loss: 0.15245389938354492 | gen_loss: -0.06989498436450958


  4%|▍         | 2/50 [00:10<03:54,  4.89s/it]

Epoch: 1 | disc_loss: 0.0009180586785078049 | gen_loss: -0.0033318146597594023


  6%|▌         | 3/50 [00:14<03:36,  4.60s/it]

...

Epoch: 48 | disc_loss: -0.08710119128227234 | gen_loss: 0.041870132088661194


100%|██████████| 50/50 [03:51<00:00,  4.62s/it]


Epoch: 49 | disc_loss: -0.10412894934415817 | gen_loss: 0.05354977026581764


Synthetic data generation: 100%|██████████| 87/87 [00:01<00:00, 69.38it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.415160,0.333333,0.136200,0.333333,0.600000,0.166667,0.307692,1.0,1.00,1.0,0.000021,0.041940,0.407321,0.375,0.0
1,0.452994,0.666667,0.128935,0.400000,0.800000,0.500000,0.846154,0.2,0.75,0.0,-0.026776,0.029268,0.403623,0.100,0.0
2,0.500516,0.166667,0.204948,0.066667,0.933333,0.666667,1.000000,0.0,0.00,1.0,-0.028636,0.001469,0.480637,0.400,0.0
3,0.208567,0.500000,0.118594,0.600000,0.800000,0.833333,0.307692,0.0,1.00,0.0,-0.027901,0.015777,0.280495,0.475,0.0
4,0.430072,0.166667,0.126853,0.666667,0.466667,0.000000,0.384615,0.0,0.00,0.0,-0.030149,0.020552,0.380429,0.650,1.0


In [12]:
eval_model(synth_data, dataset_test)

{'precision': 0.7207890743550834,
 'recall': 0.3158244680851064,
 'auroc': 0.47242836307481123,
 'dp': 0.10060642419000054,
 'ftu': 0.14}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cache/adult.npy


2022-02-04 17:11:43.219249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


Pretrain_Epoch:0, trainLoss:0.018377, validLoss:0.007400, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.007770, validLoss:0.007489, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.007857, validLoss:0.007542, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.007895, validLoss:0.007547, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.007891, validLoss:0.006571, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.002940, validLoss:0.002490, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.002588, validLoss:0.002484, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.002579, validLoss:0.002462, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.002581, validLoss:0.002461, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.002567, validLoss:0.002443, validReverseLoss:0.000000
Pretrain_Epoch:10, trainLoss:0.002573, validLoss:0.002476, validReverseLoss:0.000000
Pretrain_Epoch:11, trainLoss:0.002570, validLoss:0.002426, validReverseLoss

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000000,0.313098,0.000000,0.447853,0.307014,0.633629,0.370685,0.0,0.414991,0.0,0.000000,0.369372,0.526167,1.048100,1.0
1,0.162797,0.153582,0.032848,0.780304,0.193229,0.461950,0.398713,0.0,0.813397,0.0,0.000000,0.385241,0.000000,1.102639,1.0
2,0.000000,0.208462,0.239274,0.839121,0.000000,0.545892,0.406260,0.0,1.180958,0.0,0.000000,0.410443,0.000000,0.000000,1.0
3,0.149550,0.366827,0.022638,0.792689,0.087440,0.244400,0.490716,0.0,0.860048,0.0,0.000000,0.332419,0.000000,0.363599,1.0
4,0.063530,0.319478,0.718697,0.598897,0.166194,0.573069,0.628330,0.0,1.016383,0.0,0.070529,0.514503,0.000000,0.772208,1.0


In [14]:
eval_model(synth_data, dataset_test)

{'precision': 0.773604590505999,
 'recall': 0.9860372340425532,
 'auroc': 0.5555186170212766,
 'dp': 0.01400030693248877,
 'ftu': 0.030999999999999917}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipp

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3675,0.380142,0.000388,0.069295,0.193782,0.602936,2.099937e-01,0.196843,0.239727,6.095866e-35,1.0,2.093916e-05,1.270744e-06,0.189316,9.965676e-11,0.0
5946,0.607431,0.615952,0.023988,0.110090,0.606911,3.790743e-14,0.860790,0.708142,3.790271e-26,1.0,1.319775e-06,1.160979e-05,0.345060,2.871152e-24,0.0
16075,0.572381,0.079426,0.030153,0.208819,0.960146,4.513823e-08,0.590635,0.044224,9.959477e-27,0.0,7.349094e-08,2.085748e-02,0.393091,2.918873e-11,0.0
23510,0.084389,0.000023,0.104752,0.065256,0.816472,4.859577e-01,0.744997,0.953279,2.737193e-27,0.0,1.918518e-07,3.674373e-18,0.205421,2.281011e-12,1.0
24885,0.234341,0.000942,0.112664,0.035219,0.603397,4.412857e-03,0.835228,0.391127,7.269054e-32,1.0,3.768562e-07,8.484753e-16,0.448975,5.465361e-09,1.0


In [16]:
eval_model(synth_data, dataset_test)

{'precision': 0.8732394366197183,
 'recall': 0.7832446808510638,
 'auroc': 0.7192433081674674,
 'dp': 0.35986203,
 'ftu': 0.19800001}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3675,0.569362,2.306030e-02,0.040449,0.031290,0.460504,4.049787e-01,0.687541,0.959119,1.518811e-13,0.0,1.004490e-06,3.206931e-10,0.434427,3.572187e-23,1.0
5946,0.377764,2.334096e-05,0.122391,0.728067,0.882204,1.850907e-04,0.202852,0.380964,1.583383e-16,1.0,1.184416e-06,7.724157e-09,0.418983,5.500095e-15,1.0
16075,0.350978,6.441578e-08,0.089029,0.050665,0.487395,5.067925e-01,0.511657,0.591335,6.622320e-15,1.0,1.145748e-07,1.514143e-08,0.405432,2.919201e-19,1.0
23510,0.471133,8.222158e-06,0.122726,0.043518,0.468221,3.292640e-09,0.566924,0.369867,7.818704e-37,1.0,9.189102e-10,1.149758e-10,0.435682,8.755472e-09,0.0
24885,0.608282,1.187612e-02,0.076645,0.269258,0.461104,2.710335e-07,0.516129,0.372954,5.737523e-24,1.0,2.757823e-09,8.570197e-11,0.759713,1.737677e-05,1.0


In [18]:
eval_model(synth_data, dataset_test)

{'precision': 0.8654259126700071,
 'recall': 0.8038563829787234,
 'auroc': 0.7124120624571036,
 'dp': 0.26447117,
 'ftu': 0.009500027}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3675,0.126801,2.034548e-07,0.097748,0.445107,0.504234,5.187284e-02,0.555084,0.207682,1.940717e-26,0.0,7.969487e-07,8.796071e-22,0.438005,9.395826e-04,1.0
5946,0.481099,2.218284e-02,0.096830,0.036081,0.593112,2.401668e-04,0.822327,0.372022,3.589954e-33,1.0,3.816202e-05,3.667631e-02,0.358581,3.940624e-20,1.0
16075,0.066505,7.122482e-01,0.024837,0.168736,0.601697,3.587708e-01,0.080743,0.418794,2.489512e-37,1.0,2.710960e-05,1.951058e-18,0.418606,1.317396e-09,1.0
23510,0.091973,1.355053e-01,0.029914,0.085077,0.956460,5.039743e-07,0.500809,0.375713,1.503318e-26,1.0,4.706238e-07,1.705442e-03,0.390647,9.432316e-13,0.0
24885,0.294165,9.693924e-08,0.049272,0.744953,0.583081,7.366261e-11,0.326614,0.381308,2.713189e-27,1.0,1.116689e-09,1.143300e-22,0.427482,1.598419e-06,1.0


In [20]:
eval_model(synth_data, dataset_test)

{'precision': 0.7692307692307693,
 'recall': 0.9441489361702128,
 'auroc': 0.5426389842141386,
 'dp': 0.0018555522,
 'ftu': 0.07699996}

#### DECAF-DP

In [21]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3675,0.602783,1.143769e-07,0.040871,0.034705,0.498522,3.042221e-07,0.265838,0.692303,1.167078e-14,1.0,2.071873e-06,1.343675e-04,0.617038,2.140054e-13,1.0
5946,0.596992,2.013029e-03,0.143913,0.794298,0.455123,1.978627e-02,0.727317,0.614474,1.705038e-13,1.0,3.786253e-07,3.505098e-16,0.353130,8.698998e-23,1.0
16075,0.112791,5.396063e-07,0.082294,0.040499,0.596541,1.945614e-06,0.540141,0.381207,8.063686e-32,1.0,1.033979e-08,7.078151e-10,0.426859,2.057964e-24,0.0
23510,0.313090,4.725884e-05,0.115046,0.111139,0.596356,8.643508e-03,0.142311,0.789508,2.858072e-05,0.0,8.492114e-10,7.012393e-07,0.428549,1.228493e-14,0.0
24885,0.495689,2.521220e-06,0.031824,0.286899,0.453205,3.095134e-01,0.571484,0.978397,1.969127e-21,0.0,2.708674e-09,2.719935e-20,0.409587,2.226772e-21,0.0


In [22]:
eval_model(synth_data, dataset_test)

{'precision': 0.7696105320899616,
 'recall': 0.932845744680851,
 'auroc': 0.543035775566232,
 'dp': 0.045112073,
 'ftu': 0.021500051}